In [1]:
// https://medium.com/@mrpowers/managing-spark-partitions-with-coalesce-and-repartition-4050c57ad5c4
// from https://docs.databricks.com/notebooks/notebook-workflows.html#api

import scala.concurrent.{Future, Await}
import scala.concurrent.duration._
import scala.util.control.NonFatal

case class NotebookData(path: String, timeout: Int, parameters: Map[String, String] = Map.empty[String, String])

def parallelNotebooks(notebooks: Seq[NotebookData]): Future[Seq[String]] = {
  import scala.concurrent.{Future, blocking, Await}
  import java.util.concurrent.Executors
  import scala.concurrent.ExecutionContext
  import com.databricks.WorkflowException

  val numNotebooksInParallel = 4 
  // If you create too many notebooks in parallel the driver may crash when you submit all of the jobs at once. 
  // This code limits the number of parallel notebooks.
  implicit val ec = ExecutionContext.fromExecutor(Executors.newFixedThreadPool(numNotebooksInParallel))
  val ctx = dbutils.notebook.getContext()
  
  Future.sequence(
    notebooks.map { notebook => 
      Future {
        dbutils.notebook.setContext(ctx)
        if (notebook.parameters.nonEmpty)
          dbutils.notebook.run(notebook.path, notebook.timeout, notebook.parameters)
        else
          dbutils.notebook.run(notebook.path, notebook.timeout)
      }
      .recover {
        case NonFatal(e) => s"ERROR: ${e.getMessage}"
      }
    }
  )
}

import scala.concurrent.{Future, Await}
import scala.concurrent.duration._
import scala.util.control.NonFatal
defined class NotebookData
parallelNotebooks: (notebooks: Seq[NotebookData])scala.concurrent.Future[Seq[String]]

In [2]:
import spark.implicits._
import org.apache.spark.sql._

case class partitionToProcess(partitionKey:Int)

val ptp = Seq(
    partitionToProcess(199702),
    partitionToProcess(199703),
    partitionToProcess(199705)
)

import spark.implicits._
import org.apache.spark.sql._
defined class partitionToProcess
ptp: Seq[partitionToProcess] = List(partitionToProcess(199702), partitionToProcess(199703), partitionToProcess(199705))

In [3]:
import scala.concurrent.Await
import scala.concurrent.duration._
import scala.language.postfixOps

val notebooks = ptp.map(p => NotebookData("./partition-load-one", 180, Map("partitionKey" -> p.partitionKey.toString)))

val res = parallelNotebooks(notebooks)

Await.result(res, 180 seconds) // this is a blocking call.

res.value

import scala.concurrent.Await
import scala.concurrent.duration._
import scala.language.postfixOps
notebooks: Seq[NotebookData] = List(NotebookData(./partition-load-one,180,Map(partitionKey -> 199702)), NotebookData(./partition-load-one,180,Map(partitionKey -> 199703)), NotebookData(./partition-load-one,180,Map(partitionKey -> 199705)))
res: scala.concurrent.Future[Seq[String]] = Future(Success(List(199702, 199703, 199705)))
res0: Option[scala.util.Try[Seq[String]]] = Some(Success(List(199702, 199703, 199705)))